In [ ]:
# default_exp att_stats

# att_stats

> Attribute statistics

In [ ]:
# export
class AttStats:
    def __init__(self, statistics: list, avg_confidence: float):
        self.statistics = statistics
        self.avg_confidence = avg_confidence
        
        